In [4]:
import datetime, math
import plotly.graph_objects as go
import pandas as pd

#Swap for local file or remote version of the file
#data_path = '../COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv'
data_path = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
df = pd.read_csv(data_path)

fig = go.Figure()
#Can accept any dates in mm/dd/yy format as long as the year is between 0 and 99 (Becomes 2000 and 2099)
#If you set both dates to same date, shows total confirmed at that date rather than new cases within a period
def generate_new_map(start_date, end_date):
    total = 0
    fig.data = []
    most_recent_date = df.keys()[len(df.keys()) - 1].split('/')
    first_date = df.keys()[4].split('/')
    start_date_split = start_date.split('/')
    end_date_split = end_date.split('/')
    
    d_recent = datetime.datetime(int('20'+most_recent_date[2]),int(most_recent_date[0]),int(most_recent_date[1]))
    d_end = datetime.datetime(int('20'+end_date_split[2]),int(end_date_split[0]),int(end_date_split[1]))
    d_first = datetime.datetime(int('20'+first_date[2]),int(first_date[0]),int(first_date[1]))
    d_start = datetime.datetime(int('20'+start_date_split[2]),int(start_date_split[0]),int(start_date_split[1]))
    
    #Make sure end date always stays within the data, settings to most recent if set for the future
    if(d_end > d_recent):
        end_date = df.keys()[len(df.keys()) - 1]

    #Make sure start date always stays within the data, settings to first date if set for earlier than available
    if(d_start < d_first):
        start_date = df.keys()[4]
        
    for k,v in df.iterrows():
        val = v[end_date]
        new_cases = 0
        v['text'] = list(v['Country/Region'])
        if str(v['Province/State']) != str('nan'):
            v['text'].insert(0,str(v['Province/State']) + ", ")
        if start_date == end_date:
            new_cases = val
            v['text'].append('<br>New Cases: ' + str(new_cases))
        else:
            prev_val = v[start_date]
            
            #If data is missing, use data from other end of period, else assume no cases
            if math.isnan(val):
                if not math.isnan(prev_val):
                    val = prev_val
                else:
                    val = 0
            if math.isnan(prev_val):
                if not math.isnan(val):
                    prev_val = val
                else:
                    prev_val = 0
                    
            new_cases = int(val - prev_val)
            v['text'].append('<br>New Cases: ' + str(new_cases))
        if new_cases > 0:
            total += new_cases
            fig.add_trace(go.Scattergeo(
                lon = [v['Long']],
                lat = [v['Lat']],
                text = ''.join(v['text']),
                marker = dict(
                    size = 3,
                    color = 'crimson',
                    line_color='rgb(40,40,40)',
                    line_width=0.5,
                    sizemode = 'area'
                )))
    if start_date == end_date:
         fig.update_layout(
                title_text = 'All COVID-19 Cases ({:,} Total Cases)'.format(total),
                showlegend=False,
                geo = dict(
                    scope = 'world',
                    landcolor = 'rgb(217, 217, 217)',
                )
            )       
    else:
        fig.update_layout(
                title_text = 'New COVID-19 Cases between {0} and {1} ({2} Total New Cases)'.format(start_date,end_date,"{:,}".format(total)),
                showlegend=False,
                geo = dict(
                    scope = 'world',
                    landcolor = 'rgb(217, 217, 217)',
                )
            )
            
    fig.show()

generate_new_map('4/3/20', '4/3/20')